In [1]:
from sourmash.nodegraph import Nodegraph

In [2]:
Nodegraph?

Init signature: Nodegraph(ksize, starting_size, n_tables)
Docstring:      <no docstring>
File:           ~/anaconda3/envs/sourmash-v4.8.6/lib/python3.12/site-packages/sourmash/nodegraph.py
Type:           type
Subclasses:     

In [ ]:
def make_peptide_bloom_filter(
    peptides,
    peptide_ksize,
    molecule,
    n_tables=constants_index.DEFAULT_N_TABLES,
    tablesize=constants_index.DEFAULT_MAX_TABLESIZE,
    index_dir=None,
):
    """Create a bloom filter out of peptide sequences"""
    peptide_bloom_filter = Nodegraph(peptide_ksize, tablesize, n_tables=n_tables)
    if not index_dir:
        # if not a directory, should be single file. Convert to list to use for loop below.
        peptides = [peptides]
    for peptide_fasta in peptides:
        records = maybe_read_peptide_file(peptide_fasta)
        for record in tqdm(records):
            if "*" in record["sequence"]:
                continue
            sequence = encode_peptide(record["sequence"], molecule)
            try:
                kmers = kmerize(sequence, peptide_ksize)
                for kmer in kmers:
                    # Convert the k-mer into an integer
                    hashed = hash_murmur(kmer)

                    # .add can take the hashed integer so we can hash the
                    #  peptide kmer and add it directly
                    peptide_bloom_filter.add(hashed)
            except ValueError:
                # Sequence length is smaller than k-mer size
                continue
    return peptide_bloom_filter